In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage
import math

In [ ]:
X = (np.array(skimage.io.imread_collection("CIFAR10\\train\\*")) / 255)[:5000]
#Y = (pd.read_csv("CIFAR10/trainLabels.csv").label.to_numpy())[:5000]
y1h = (pd.get_dummies(pd.read_csv("CIFAR10/trainLabels.csv").label).to_numpy())[:5000]
Y = np.where(y1h == 1)[1]
print(X.shape, Y.shape, y1h.shape)

(5000, 32, 32, 3) (5000,) (5000, 10)


In [ ]:
def score(X, w, b):
    return X.dot(w) + b
def softmax(x):
    e = np.exp(x)
    return e/np.sum(e, axis=1, keepdims=True)
def RELU(x):
    return np.maximum(0, x)
def ReLU_deriv(x):
    return x > 0
def lossSoftmax(softmaxScore, y1h):
    a = np.multiply(softmaxScore, y1h)
    b = np.sum(a,axis=1, keepdims=True)
    c = -np.log(b)
    return np.mean(c)
def accuracyOfModel(score, Y):
    return np.mean(np.argmax(score, axis=1) == Y)
def convXF(x, f):
    cln = []
    m, n, _ = x.shape
    o = f.shape[0]
    for m in range(m):
        tmp = []
        for i in range(n-o+1):
            for j in range(n-o+1):
                tmp.append(x[m][i:i+o, j:j+o].flatten().dot(f.flatten().T))
        cln.append(tmp)
    return np.array(cln).reshape(m+1, n-o+1, n-o+1)
def conv3D(X, f):
    m, n, o, p = X.shape
    q, r, s, t = f.shape
    cln = []
    for m in range(m):
        tmp1 = []
        for i in range(n-q+1):
            for j in range(n-q+1):
                tmp2 = []
                for k in range(t):
                    tmp2.append(X[m][i:i+q, j:j+q, :].flatten().dot(f[:,:,:,k].flatten().T))
                tmp1.append(tmp2)
        cln.append(tmp1)
    return np.array(cln).reshape(len(cln), int(math.sqrt(len(cln[0]))), int(math.sqrt(len(cln[0]))), len(cln[0][0]))
def maxPool(z, o):
    m, n, _ = z.shape
    pl=[]
    for m in range(m):
        tmp = []
        for i in range(0, n, o):
            for j in range(0, n, o):
                tmp.append(np.max(z[m][i:i+o, j:j+o]))
        pl.append(tmp)
    return np.array(pl).reshape(len(pl), int(n/o), int(n/o))
def maxPool3D(z, o):
    lst = []
    for i in range(z.shape[3]):
        lst.append(maxPool(z[:,:,:,i],o))
    return np.array(lst).swapaxes(0,1).swapaxes(1,2).swapaxes(2,3)
def maxIndices(x):
    a = np.argmax(x)
    b = x.shape[1]
    indices = (int(a/b), int(a%b))
    return indices
def backPropMaxPool(x, p, o):
    z_t = np.zeros(x.shape)
    m, n, _ = z_t.shape
    for m in range(m):
        for i in range(0, n, o):
            for j in range(0, n, o):
                maxI = maxIndices(x[m][i:i+o, j:j+o])
                z_t[m][i+maxI[0], j+maxI[1]] = p[i,j]
    return z_t
def backPropMaxPool3D(x,p,o):
    lst = []
    for i in range(x.shape[-1]):
        lst.append(backPropMaxPool(x[:,:,:,i], p, o))
    return np.array(lst).swapaxes(0,1).swapaxes(1,2).swapaxes(2,3)
def duplicateAlongChannels(x, n):
    lst = []
    for i in range(n):
        lst.append(x)
    return np.array(lst).swapaxes(0,1).swapaxes(1,2)

## Forward Pass

In [ ]:
f = np.random.randn(3,3,3,5) * 0.5
print(X.shape, f.shape)

(5000, 32, 32, 3) (3, 3, 3, 5)


In [ ]:
z1 = conv3D(X, f)

In [ ]:
z1.shape

(5000, 30, 30, 5)

In [ ]:
p1 = maxPool3D(z1, 2)

In [ ]:
p1.shape

(5000, 15, 15, 5)

In [ ]:
p1f = p1.reshape(p1.shape[0], (p1.shape[1]*p1.shape[2]*p1.shape[3]))

In [ ]:
p1f.shape

(5000, 1125)

In [ ]:
m, n = p1f.shape
c = y1h.shape[1]
w2 = 0.05 * np.random.randn(n, c)
b2 = 0.05 * np.random.randn(1, c)

In [ ]:
z2 = score(p1f, w2, b2)
a2 = softmax(z2)
loss = lossSoftmax(a2, y1h)

In [ ]:
loss

3.2547534485660856

## Backward pass

In [ ]:
#Gradients
#Softmax Layer
dz2 = (1/m) * (a2 - y1h)
dw2 = p1f.T.dot(dz2)
db2 = np.sum(dz2, axis=0, keepdims=True)

In [ ]:
# Conv Layer
dp1 = dz2.dot(w2.T)
dp1_b =  np.sum(backPropMaxPool3D(z1, dp1, 2), axis=0)
dp1_bn = duplicateAlongChannels(dp1_b, X.shape[-1])
drf = (1/m) * np.sum(conv3D(X, dp1_bn), axis=0)

## Gradient Decent

In [ ]:
f = 0.5 * np.random.randn(3,3,3,5) 
w2 = 0.05 * np.random.randn(n, c)
b2 = 0.05 * np.random.randn(1, c)

In [ ]:
lr = 0.0001
itr = 100
for i in range(itr+1):
    #Forward Pass
    #Conv layer
    z1 = conv3D(X, f)
    p1 = maxPool3D(z1, 2)
    #Softmax Layer
    p1f = p1.reshape(p1.shape[0], (p1.shape[1]*p1.shape[2]*p1.shape[3]))
    m, n = p1f.shape
    c = y1h.shape[1]
    
    z2 = score(p1f, w2, b2)
    a2 = softmax(z2)
    loss = lossSoftmax(a2, y1h)
    
    #Gradients
    #Softmax Layer
    dz2 = (1/m) * (a2 - y1h)
    dw2 = p1f.T.dot(dz2)
    db2 = np.sum(dz2, axis=0, keepdims=True)

    # Conv Layer
    dp1 = dz2.dot(w2.T)
    dp1_b =  np.sum(backPropMaxPool3D(z1, dp1, 2), axis=0)
    dp1_bn = duplicateAlongChannels(dp1_b, X.shape[-1])
    drf = (1/m) * np.sum(conv3D(X, dp1_bn), axis=0)
    
    #Weights and Bias Update
    f -= lr*drf
    w2 -= lr*dw2
    b2 -= lr*db2
    print(i, loss, accuracyOfModel(a2, Y))

0 2.2735385206906966 0.2022
1 2.2695746901902507 0.2022
2 2.2657684931134208 0.2028
3 2.262112876163554 0.2028
4 2.258601181578455 0.2044
5 2.2552270794384697 0.204
6 2.251984621000697 0.2024
7 2.248868193100175 0.2026
8 2.2458724963924803 0.2036
9 2.2429924544222852 0.2038
10 2.2402232861514983 0.2052
11 2.2375604140378456 0.2058
12 2.234999537004221 0.2054
13 2.232536516834076 0.2052
14 2.2301673967263325 0.2058
15 2.2278883651992385 0.207
16 2.2256958217366187 0.2062
17 2.2235863236768716 0.2068
18 2.221556574488815 0.2072
19 2.2196034085196974 0.2078
20 2.217723830774711 0.2066
21 2.2159149304017993 0.207
22 2.2141739835634042 0.2072
23 2.2124983033440775 0.2076
24 2.2108853663341788 0.208
25 2.20933270298611 0.2084
26 2.2078380187377276 0.2084
27 2.206399012864216 0.2096
28 2.205013542304435 0.2086
29 2.203679514453898 0.208
30 2.2023949490274317 0.208
31 2.2011578930898636 0.2088
